In [1]:
API_KEY = open("key.txt", "r").read().replace("\n","")

In [2]:
# as per https://developers.google.com/places/web-service/search
MAX_RADIUS      = 25000  #in meters
MAX_RESULT_SIZE = 20
MAX_PAGE_COUNT  = 3
DEFAULT_SLEEP   = 2
THREAD_COUNT    = 16
MAX_JOBS        = 85 # 1 + 4 + 16 + 64 

In [3]:
import requests
import pandas as pd 
from pandas.io.json import json_normalize
import json
import time
import numpy as np
import queue
import threading
import datetime
import geopy
import geopy.distance
from geopy.distance import VincentyDistance

%matplotlib inline

In [4]:
city_centers ={ "Riyadh":(24.713600, 46.675300),
                "Jeddah":(21.485800, 39.192500),
                "Makkah": (21.389100, 39.857900),
                "Madinah": (24.524700, 39.569200),
                "Dammam":(26.420700, 50.088800),
                "Khobar":(26.217200, 50.197100)}

In [5]:
categories =['bank','bakery','cafe','gym','mosque','park','real_estate_agency','shopping_mall','supermarket','lodging','laundry','gas_station','convenience_store']

In [38]:
category = categories[12]
category

'convenience_store'

In [39]:
completed_jobs = []
enqueued = []
dfs = []
job_queue = queue.Queue()

In [40]:
#return centers and radius of minimal circles containing quarters of a minimal square containing passed circled as defined by its center and radius
def get_smaller_circles(location,radius):
# given: lat, lon, b = bearing in degrees, d = distance in meters
    origin = geopy.Point(location)
    origins=[]
    radius=radius*2**0.5/2
    for i in range(4):
        destination = geopy.distance.geodesic(meters=radius).destination(origin, 90*i+45)
        origins.append((destination.latitude, destination.longitude))
    return origins,radius


In [41]:
def add_job(paramters,job_queue):
    print("adding job location {} radius {}".format(paramters['location'],paramters['radius']))
    enqueued.append(paramters)
    job_queue.put(paramters)

def get_job(job_queue):
    return job_queue.get()

In [42]:
def find_place_from_text(query,location,radius=MAX_RADIUS,api_key=API_KEY,sleep_time=DEFAULT_SLEEP,start_time=None,start_location=None,recursion=False):
    
    results = []
    next_page_token = True
    next_page_count = 0
    
    while next_page_token:
    
        #1 you need to put your google key 
        # https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Museum%20of%20Contemporary%20Art%20Australia&inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry&key=YOUR_API_KEY
        # query = urlencode.encode('Musuem Contemporty')
        if len(results)==0:
            next_token = ''
        else:
            next_token = '&pagetoken=' + next_page_token
            next_page_count+=1

        #fields = ['photos','formatted_address','name','opening_hours','rating']
        
        result = requests.get("https://maps.googleapis.com/maps/api/place/textsearch/json?query={query_text}&inputtype=textquery&key={query_api_key}&location={query_location}&radius={query_radius}&rank_by={query_rank_by}&type={query_type}{next_token}".format(\
            query_api_key=api_key,
            query_text= query,
            query_location=','.join([str(x) for x in location]),
            query_radius=radius,
            query_rank_by="distance",
            query_type=query,
            next_token=next_token
        ))

        # turn response to json
        google_result = result.json()
        #collect next page token
        next_page_token = google_result.get('next_page_token', None)

        # get results field
        #results = google_result["results"]
        # update results count
        
        # make df from results
        results.append(pd.DataFrame.from_dict(google_result["results"], orient='columns'))

        # add time delay because google is stupid
        
        if(next_page_token):
            time.sleep(sleep_time)
    df=pd.concat([*results]).reset_index(drop=True)   
    return df ,next_page_count

In [43]:
#df, count =find_place_from_text("bla",city_centers['Riyadh'],radius=100,api_key=API_KEY,sleep_time=DEFAULT_SLEEP)
#df

In [44]:
#print(df.geometry[0])
#geopy.distance.geodesic(city_centers['Riyadh'],(12.9549865,-5.7563151)).km

In [45]:
!ls *csv

bakery_riyadh_25km.csv             mosque_riyadh_25km.csv
bank_riyadh_25km.csv               park_riyadh_25km.csv
cafe_riyadh_25km.csv               real_estate_agency_riyadh_25km.csv
gas_station_riyadh_25km.csv        restaurants_riyadh_25km.csv
gym_riyadh_25km.csv                riyadh.csv
hospitals_riayd.csv                riyadh_land.csv
hospitals_small_radius_riayd.csv   schools_riayd.csv
laundry_riyadh_25km.csv            shopping_mall_riyadh_25km.csv
listings.csv                       supermarket_riyadh_25km.csv
lodging_riyadh_25km.csv            tempdf.csv


In [46]:
def worker():
    #To end threads restart kernal. Careful: export your resutls into a csv file
    while True:
        item = get_job(job_queue)
        if item is None:
            break
        df,next_page_count = find_place_from_text(**item)
        size=len(df)
        item["compeleted"]=datetime.datetime.now()
        item["size"]=size
        item["next_page_count"]=next_page_count
        item["distance_from_start_point"]=geopy.distance.geodesic(item['start_location'],item['location']).m
        completed_jobs.append(item)
        
        if(size==MAX_PAGE_COUNT*MAX_RESULT_SIZE and len(enqueued)<MAX_JOBS and item['recursion']==True):
            origins,radius = get_smaller_circles(item['location'],item['radius'])
            for origin in origins:
                job={'query':item['query'],
                'location': origin,
                'radius':radius,
                'api_key':item['api_key'],
                'start_time':datetime.datetime.now(),
                'start_location': item['start_location'],
                'sleep_time':item['sleep_time'],
                'recursion':True}
                add_job(job,job_queue)
            
        dfs.append(df)
        #q.task_done()


threads = []
for i in range(THREAD_COUNT):
    t = threading.Thread(target=worker)
    t.start()
    threads.append(t)

In [47]:
job={'query':category,
     'location': city_centers['Riyadh'],
     'radius':MAX_RADIUS,
     'api_key':API_KEY,
     'start_time':datetime.datetime.now(),
     'start_location': city_centers['Riyadh'],
     'sleep_time':DEFAULT_SLEEP,
     'recursion':True}

add_job(job,job_queue)

adding job location (24.7136, 46.6753) radius 25000


In [63]:
len(completed_jobs)

85

In [64]:
df = pd.concat([*dfs]).reset_index(drop=True) 

In [65]:
len(df),len(df.name.unique())

(5038, 98)

In [66]:
df['category']=category
#df.head()

In [67]:
df.drop_duplicates(['name']).to_csv('{}_riyadh_25km.csv'.format(category),index=False)

In [68]:
df = pd.read_csv('{}_riyadh_25km.csv'.format(category))
df

,formatted_address,geometry,icon,id,name,opening_hours,permanently_closed,photos,place_id,plus_code,price_level,rating,reference,types,user_ratings_total,category
0,الملز، الرياض 12641,"{'location': {'lat': 24.6494352, 'lng': 46.724...",https://maps.gstatic.com/mapfiles/place_api/ic...,a62ecf46f3aebdb1130bd2373c6b4af4b5363118,Circle K - Al Malaz,{'open_now': False},NaN,"[{'height': 3836, 'html_attributions': ['<a hr...",ChIJqV14e0sELz4RlWwyzz4ANDI,"{'compound_code': 'JPXF+QJ الملز، الرياض', 'gl...",1.0,5.0,ChIJqV14e0sELz4RlWwyzz4ANDI,"['convenience_store', 'store', 'point_of_inter...",4,convenience_store
1,شارع الشباب، حي العزيزية، العزيزية، الرياض 14514,"{'location': {'lat': 24.5832125, 'lng': 46.763...",https://maps.gstatic.com/mapfiles/place_api/ic...,569cd3015563692b4bf8e709d33354b1d03ae417,أسواق فهد المركزية للمواد الغذائية,{'open_now': False},NaN,NaN,ChIJt17Y4yAJLz4RwMrpttfw-N4,"{'compound_code': 'HQM7+7H العزيزية، الرياض', ...",NaN,3.0,ChIJt17Y4yAJLz4RwMrpttfw-N4,"['convenience_store', 'grocery_or_supermarket'...",1,convenience_store
2,المغيرة بن شعبة، Al Mughirah Ibn Shubah، الريا...,"{'location': {'lat': 24.7479012, 'lng': 46.667...",https://maps.gstatic.com/mapfiles/place_api/ic...,5e5db63d656bf4c9e694d409639aac3d74c44697,وكالة أفكار الإعلام للدعاية والاعلان,{'open_now': False},NaN,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJfQhT4ariLj4RLUK1fLXfT8Y,"{'compound_code': 'PMX8+5X الملك فهد، الرياض',...",NaN,3.9,ChIJfQhT4ariLj4RLUK1fLXfT8Y,"['convenience_store', 'supermarket', 'grocery_...",13,convenience_store
3,أبي الأسود الدؤلي، النسيم، الرياض 14242,"{'location': {'lat': 24.7521109, 'lng': 46.835...",https://maps.gstatic.com/mapfiles/place_api/ic...,8388a62cf96ec687f731c5b3f3f40b366fb41afe,تموينات ابراهيم عبد الله ابراهيم بن سعيد,{'open_now': False},NaN,NaN,ChIJz23112-qLz4R42jhHKeyPbA,{'compound_code': 'QR2P+R2 النسيم الشرقي، الري...,NaN,0.0,ChIJz23112-qLz4R42jhHKeyPbA,"['convenience_store', 'grocery_or_supermarket'...",0,convenience_store
4,الرياض 13782,"{'location': {'lat': 24.6314432, 'lng': 46.562...",https://maps.gstatic.com/mapfiles/place_api/ic...,c553eff314cdab17c34708c182e5bf5b9f6a2996,أسواق عال الكيف,NaN,NaN,NaN,ChIJNTXcQNIeLz4RvKBwsOAV7p8,"{'compound_code': 'JHJ7+H5 ظهرة لبن، الرياض', ...",NaN,4.0,ChIJNTXcQNIeLz4RvKBwsOAV7p8,"['convenience_store', 'supermarket', 'grocery_...",5,convenience_store
5,شارع الرشد، حي المونسية، المونسية، الرياض 13246,"{'location': {'lat': 24.8232661, 'lng': 46.753...",https://maps.gstatic.com/mapfiles/place_api/ic...,51eadddfd9acb1087bbe72ec0bae5897ccafc655,أسواق ومخابز أفنان الغذاء,{'open_now': True},NaN,NaN,ChIJJyyoopv-Lj4Rh9QNIX-PhX8,"{'compound_code': 'RQF3+8H المونسية، الرياض', ...",NaN,3.7,ChIJJyyoopv-Lj4Rh9QNIX-PhX8,"['convenience_store', 'grocery_or_supermarket'...",3,convenience_store
6,تبوك، ظهرة لبن، الرياض 13784,"{'location': {'lat': 24.6335662, 'lng': 46.535...",https://maps.gstatic.com/mapfiles/place_api/ic...,b95a7711438b60e8200ee442c01c7201c6843132,اسواق ثمار ليمونة,{'open_now': True},NaN,"[{'height': 1265, 'html_attributions': ['<a hr...",ChIJ7YlPn0AfLz4R_Kr1cmE3b54,"{'compound_code': 'JGMP+C5 ظهرة لبن، الرياض', ...",NaN,3.9,ChIJ7YlPn0AfLz4R_Kr1cmE3b54,"['convenience_store', 'store', 'point_of_inter...",46,convenience_store
7,"طريق النهضة، Ar Rabwah, الرياض 12813","{'location': {'lat': 24.7010963, 'lng': 46.746...",https://maps.gstatic.com/mapfiles/place_api/ic...,43c87a8cc1ea03b18921b4e8a21c839a4c0ded3c,اي دي سوبرماركت,{'open_now': False},NaN,"[{'height': 3264, 'html_attributions': ['<a hr...",ChIJpZVeqP0DLz4R8dLR70k7vaU,"{'compound_code': 'PP2W+CC الربوة، الرياض', 'g...",NaN,4.3,ChIJpZVeqP0DLz4R8dLR70k7vaU,"['convenience_store', 'store', 'point_of_inter...",3,convenience_store
8,الطريق الدائري الشرقي، الريان، الرياض 14211,"{'location': {'lat': 24.7067133, 'lng': 46.768...",https://maps.gstatic.com/mapfiles/place_api/ic...,1b71344317807cccca25e8681d584a380caab098,اي دي ماركت,{'open_now': False},NaN,"[{'height': 3968, 'html_attributions': ['<a hr...",C

In [69]:
#len(pd.concat([hospitals,hospitals2]).drop_duplicates(['name']))

In [38]:
#riyadh = pd.read_csv('riyadh.csv')

In [39]:
#for location in riyadh.sample(n=100,random_state=42).location_str:
#    job={'query':'hospital',
#     'location': [float(cor) for cor in location.split(',')],
#     'radius':MAX_RADIUS,
#     'api_key':API_KEY,
#     'start_time':datetime.datetime.now(),
#     'start_location': city_centers['Riyadh'],
#     'sleep_time':DEFAULT_SLEEP,
#     'recursion':False}
#    add_job(job,job_queue)
#    print(location)

In [40]:
#completed_jobs

In [41]:
#gmap = gmplot.GoogleMapPlotter(*city_centers['Riyadh'], 12)

In [10]:
import gmaps
import gmaps.datasets
from ipywidgets.embed import embed_minimal_html
gmaps.configure(api_key=API_KEY) # Your Google API key

df = gmaps.datasets.load_dataset_as_df('starbucks_kfc_uk')

starbucks_df = df[df['chain_name'] == 'starbucks']
starbucks_df = starbucks_df[['latitude', 'longitude']]

starbucks_layer = gmaps.symbol_layer(
    starbucks_df, fill_color="green", stroke_color="green", scale=2
)
fig = gmaps.figure()
fig.add_layer(starbucks_layer)
embed_minimal_html('export.html', views=[fig])

In [11]:
!ls|grep html

export.html
my_map.html
riyadh_map.html


In [8]:
type(fig)

gmaps.figure.Figure

In [9]:

import gmaps

gmaps.configure(api_key="AI...")

fig = gmaps.figure()


<bound method Widget.open of Figure(layout=FigureLayout(height='420px'))>